In [7]:
import pandas as pd
# Lee el archivo CSV en un DataFrame de pandas
df = pd.read_csv("dataset_entrenamiento.csv",thousands=',')
# Obtén 1000 filas aleatorias del DataFrame
random_rows = df.sample(n=1000, random_state=42)
# Encuentra las clases únicas en la columna "clase"
unique_classes = random_rows["target"].unique()
unique_classes

array(['cultura', 'deportes', 'medioambiente', 'sociedad',
       'cienciatecnologia', 'salud', 'internacional', 'economia',
       'politica', 'entretenimiento'], dtype=object)

In [8]:
# Crea la lista de tuplas con las etiquetas de clases
data_list = []
for _, row in random_rows.iterrows():
    title = row["title"]
    clase = row["target"]
    cats = {cls: 1 if cls == clase else 0 for cls in unique_classes}
    data_list.append((title, {"cats": cats}))
data_list

[('Vargas Llosa ingresa en la Academia Francesa como ‘inmortal’: «La literatura necesita libertad para existir»',
  {'cats': {'cultura': 1,
    'deportes': 0,
    'medioambiente': 0,
    'sociedad': 0,
    'cienciatecnologia': 0,
    'salud': 0,
    'internacional': 0,
    'economia': 0,
    'politica': 0,
    'entretenimiento': 0}}),
 ('Rockódromo 2023: ¿Cuándo es y qué artistas tocan en el festival gratuito?',
  {'cats': {'cultura': 1,
    'deportes': 0,
    'medioambiente': 0,
    'sociedad': 0,
    'cienciatecnologia': 0,
    'salud': 0,
    'internacional': 0,
    'economia': 0,
    'politica': 0,
    'entretenimiento': 0}}),
 ('Famoso barítono argentino murió en medio de un concierto en Francia',
  {'cats': {'cultura': 1,
    'deportes': 0,
    'medioambiente': 0,
    'sociedad': 0,
    'cienciatecnologia': 0,
    'salud': 0,
    'internacional': 0,
    'economia': 0,
    'politica': 0,
    'entretenimiento': 0}}),
 ('El Barça podría quedar fuera de la Copa por un ‘caso Cheryshev

In [9]:
import spacy
import random
from spacy.training.example import Example
from spacy.util import minibatch

from spacy.pipeline.tok2vec import DEFAULT_TOK2VEC_MODEL

# Datos de entrenamiento
TRAIN_DATA = data_list
# Inicializar spaCy con el modelo de lenguaje es_core_news_sm
nlp = spacy.load("es_core_news_sm")
train_examples = []

for example in TRAIN_DATA:
    train_examples.append(Example.from_dict(nlp.make_doc(example[0]), example[1]))

def get_examples():
    return train_examples
model = {
            "@architectures": "spacy.TextCatCNN.v2",
            "exclusive_classes": True,
            "tok2vec": DEFAULT_TOK2VEC_MODEL,
        }
# Crear un componente TextCategorizer con un modelo CNN
textcat = nlp.add_pipe("textcat", config={"model": model})
textcat.initialize(get_examples)
from spacy.util import compounding

# Entrenar el modelo
with nlp.select_pipes(enable="textcat"):
    optimizer = nlp.begin_training()
    for epoch in range(100):
        losses = {}
        random.shuffle(TRAIN_DATA)
        # Dividir los datos en lotes y actualizar el modelo
        for batch in minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001)):
            texts, annotations = zip(*batch)
            example = []
            # Actualizar el modelo con iteraciones
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
            nlp.update(example, drop=0.5, losses=losses)
        print(losses)

{'textcat': 22.018773935735226}
{'textcat': 21.222719088196754}
{'textcat': 18.87881139665842}
{'textcat': 15.429258489981294}
{'textcat': 12.619538441766053}
{'textcat': 9.560523001884576}
{'textcat': 8.116677957368665}
{'textcat': 5.927624078872213}
{'textcat': 5.1587848147616455}
{'textcat': 3.9786509241694965}
{'textcat': 2.953307611431242}
{'textcat': 2.647719293283796}
{'textcat': 2.120369964597661}
{'textcat': 1.803391057268058}
{'textcat': 1.5342998595521944}
{'textcat': 1.557528163941923}
{'textcat': 1.0214065467162177}
{'textcat': 1.2256382847533618}
{'textcat': 0.8064659690463529}
{'textcat': 0.47755446181976563}
{'textcat': 0.9880148992797491}
{'textcat': 0.6893932970780136}
{'textcat': 0.6161589305684086}
{'textcat': 0.8020614881588712}
{'textcat': 0.8241045036452351}
{'textcat': 0.5733564536055568}
{'textcat': 0.5027940503442269}
{'textcat': 0.7212226094355279}
{'textcat': 0.7726589907062664}
{'textcat': 0.5808328088804908}
{'textcat': 0.5458753948795287}
{'textcat': 0.73

In [10]:
# Guardar el modelo entrenado en disco
nlp.to_disk("model_1000_CNNv2")



In [11]:
# Cargar el modelo entrenado
nlp_loaded = spacy.load("model_1000_CNNv2")#modelo_clasificador_noticias")


In [12]:
# Realizar predicciones en nuevos titulares
nuevos_titulares = [
    "El partido de fútbol se jugará mañana",
    "Nueva legislación de impuestos aprobada por el gobierno",
    "Nueva aplicación móvil para seguimiento de ejercicios",
    "Grave accidente de tráfico en la autopista deja varios heridos",
    "Avión se estrella en zona montañosa, sobreviven todos los pasajeros",
    "Incendio en fábrica causa evacuación masiva en la ciudad",
    "Naufragio de embarcación turística en el mar, varios desaparecidos",
    "Accidente de tren en la estación central, servicios suspendidos",
    "Nuevas regulaciones para reducir las emisiones de gases contaminantes en la industria",
    "Los científicos advierten sobre el aumento del nivel del mar debido al deshielo de los polos",
    "Proyecto de reforestación masiva busca combatir la deforestación en la Amazonia",
    "La contaminación del aire en las ciudades alcanza niveles peligrosos para la salud",
    "Descubren una nueva especie en peligro de extinción en la selva tropical",
    "Acuerdo internacional para proteger los océanos y conservar la vida marina",
    "Innovadora tecnología de reciclaje convierte plásticos en materiales reutilizables",
    "Cambio climático: las temperaturas extremas se vuelven más frecuentes y devastadoras",
    "Los esfuerzos de conservación salvan a una especie en peligro crítico de desaparecer",
    "La energía solar y eólica supera a los combustibles fósiles como fuente de electricidad",
]

for titular in nuevos_titulares:
    doc = nlp_loaded(titular)
    print(f"Titular: {titular}")
    print("Categorías predichas:")
    for label, score in doc.cats.items():
        print(f"{label}: {score:.4f}")

Titular: El partido de fútbol se jugará mañana
Categorías predichas:
cultura: 0.0000
deportes: 1.0000
medioambiente: 0.0000
sociedad: 0.0000
cienciatecnologia: 0.0000
salud: 0.0000
internacional: 0.0000
economia: 0.0000
politica: 0.0000
entretenimiento: 0.0000
Titular: Nueva legislación de impuestos aprobada por el gobierno
Categorías predichas:
cultura: 0.0000
deportes: 0.0000
medioambiente: 0.0000
sociedad: 0.0183
cienciatecnologia: 0.0017
salud: 0.0000
internacional: 0.0000
economia: 0.0000
politica: 0.9799
entretenimiento: 0.0000
Titular: Nueva aplicación móvil para seguimiento de ejercicios
Categorías predichas:
cultura: 0.0000
deportes: 0.0000
medioambiente: 0.0000
sociedad: 0.0001
cienciatecnologia: 0.0000
salud: 0.0000
internacional: 0.0000
economia: 0.0000
politica: 0.0000
entretenimiento: 0.9999
Titular: Grave accidente de tráfico en la autopista deja varios heridos
Categorías predichas:
cultura: 0.0000
deportes: 1.0000
medioambiente: 0.0000
sociedad: 0.0000
cienciatecnologia